# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

In [1]:
### Added by Mahbod, for imports
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [2]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader("./CONLL2003", 'train.txt', ['words', 'pos', 'ignore', 'chunk']) # if you pull from the repo, path shall be fine

training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
   # print(token, pos, ne_label)
   features_dict = {
      'words': token, 'pos': pos
   }
   training_features.append(features_dict)
   training_gold_labels.append(ne_label)
   
print("Sample Training Features:", training_features[:2])
print("Sample Training Labels:", training_gold_labels[:2])

Sample Training Features: [{'words': 'EU', 'pos': 'NNP'}, {'words': 'rejects', 'pos': 'VBZ'}]
Sample Training Labels: ['B-ORG', 'O']


In [3]:
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader("./CONLL2003", 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

test_features = []
test_gold_labels = []
for token, pos, ne_label in train.iob_words():
    # print(token, pos, ne_label)
    features_dict = {
        'words': token, 'pos': pos
    }
    test_features.append(features_dict)
    test_gold_labels.append(ne_label)

print("Sample Test Features:", test_features[:2])
print("Sample Test Labels:", test_gold_labels[:2])

Sample Test Features: [{'words': 'SOCCER', 'pos': 'NN'}, {'words': '-', 'pos': ':'}]
Sample Test Labels: ['O', 'O']


**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [4]:
from collections import Counter 

my_list=[1,2,1,3,2,5]
Counter(my_list)


Counter({1: 2, 2: 2, 3: 1, 5: 1})

In [5]:
num_train_instances = len(training_features)
num_test_instances = len(test_features)

train_label_distribution = Counter(training_gold_labels)
test_label_distribution = Counter(test_gold_labels)
train_label_df = pd.DataFrame(train_label_distribution.items(), columns=['Label', 'Frequency']).sort_values(by='Frequency', ascending=False)
test_label_df = pd.DataFrame(test_label_distribution.items(), columns=['Label', 'Frequency']).sort_values(by='Frequency', ascending=False)

print(f"Number of instances in training data: {num_train_instances}")
print(f"Number of instances in test data: {num_test_instances}")


Number of instances in training data: 203621
Number of instances in test data: 46435


In [6]:
print("Training Data - NERC Label Frequency")
print(train_label_df)
print("---")
print("Test Data - NERC Label Frequency")
print(test_label_df)

Training Data - NERC Label Frequency
    Label  Frequency
1       O     169578
5   B-LOC       7140
3   B-PER       6600
0   B-ORG       6321
4   I-PER       4528
6   I-ORG       3704
2  B-MISC       3438
8   I-LOC       1157
7  I-MISC       1155
---
Test Data - NERC Label Frequency
    Label  Frequency
0       O      38323
1   B-LOC       1668
7   B-ORG       1661
2   B-PER       1617
3   I-PER       1156
8   I-ORG        835
5  B-MISC        702
4   I-LOC        257
6  I-MISC        216


In [7]:
train_total_labels = sum(train_label_distribution.values())
test_total_labels = sum(test_label_distribution.values())

train_balance = {label: round((count / train_total_labels) * 100, 2) for label, count in train_label_distribution.items()}
test_balance = {label: round((count / test_total_labels) * 100, 2) for label, count in test_label_distribution.items()}
train_balance_df = pd.DataFrame(train_balance.items(), columns=['Label', 'Percentage']).sort_values(by='Percentage', ascending=False)
test_balance_df = pd.DataFrame(test_balance.items(), columns=['Label', 'Percentage']).sort_values(by='Percentage', ascending=False)

print("Training Data - Label Distribution (%)")
print(train_balance_df)
print("---")
print("Test Data - Label Distribution (%)")
print(test_balance_df)

Training Data - Label Distribution (%)
    Label  Percentage
1       O       83.28
5   B-LOC        3.51
3   B-PER        3.24
0   B-ORG        3.10
4   I-PER        2.22
6   I-ORG        1.82
2  B-MISC        1.69
7  I-MISC        0.57
8   I-LOC        0.57
---
Test Data - Label Distribution (%)
    Label  Percentage
0       O       82.53
1   B-LOC        3.59
7   B-ORG        3.58
2   B-PER        3.48
3   I-PER        2.49
8   I-ORG        1.80
5  B-MISC        1.51
4   I-LOC        0.55
6  I-MISC        0.47


**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [8]:
from sklearn.feature_extraction import DictVectorizer

In [9]:
vec = DictVectorizer()
all_features = training_features + test_features
the_array = vec.fit_transform(all_features).toarray()

# print(the_array.shape)

train_features_array = the_array[:num_train_instances]
test_features_array = the_array[num_train_instances:]

print("Shape of training features array:", train_features_array.shape)
print("Shape of test features array:", test_features_array.shape)

Shape of training features array: (203621, 27361)
Shape of test features array: (46435, 27361)


**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [10]:
from sklearn import svm

In [ ]:
lin_clf = svm.LinearSVC(max_iter=10000) # max_iter added to remove the problems related to convergence, 5k didnt work.
# To remove the convergence error we need to increase the maximum number of iterations, but the time goes up with it.


In [16]:
##### [ YOUR CODE SHOULD GO HERE ]
lin_clf.fit(train_features_array, training_gold_labels)

test_predictions = lin_clf.predict(test_features_array)
report = classification_report(test_gold_labels, test_predictions)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

       B-LOC       0.81      0.78      0.79      1668
      B-MISC       0.78      0.66      0.72       702
       B-ORG       0.79      0.52      0.63      1661
       B-PER       0.86      0.44      0.58      1617
       I-LOC       0.62      0.53      0.57       257
      I-MISC       0.57      0.59      0.58       216
       I-ORG       0.70      0.47      0.56       835
       I-PER       0.33      0.87      0.48      1156
           O       0.98      0.98      0.98     38323

    accuracy                           0.92     46435
   macro avg       0.72      0.65      0.65     46435
weighted avg       0.94      0.92      0.92     46435



**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [13]:
import pandas

In [14]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/Users/piek/Desktop/ONDERWIJS/data/nerc_datasets/kaggle/ner_v2.csv'
kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook